# Superposed Epoch Analysis CMIP6 Historical model runs
- opens all the CMIP6 historical model run files with data on precipitation pr and surface temperature tas
- calculates monthly anomalies (based on 5-yr climatology) for 4 spatial regions (global, SH, Aus, E Aus) and 5 eruptions
- calculates percentiles (10, 90) to show model spread
- superposes the time-series for 5 eruptions plus mmm for tas and pr and different regions (superposed epoch analysis)

In [1]:
import xarray as xr, matplotlib.pyplot as plt
from importlib import reload # need to use this if I edit a function file
import os
import numpy as np
import pandas as pd
import cartopy.crs as ccrs # to add in continents and change map projections 
from matplotlib.colors import LinearSegmentedColormap # to change colour bar????
import seaborn as sns
import dask.diagnostics # dask allows you to check how long something is taking to load

In [2]:
# import custom functions
import sys 
sys.path.append('/home/563/kb6999/Functions') # use this if the function file is in a different directory to the notebook

import frequently_used_functions as func
import plotting_functions as fplot
import model_functions as funcM
import reanalysis_functions as funcR

In [3]:
import dask.distributed
import tempfile

tempdir = tempfile.TemporaryDirectory("dask-worker-space")
dask.distributed.Client(local_directory=tempdir.name, memory_limit='16gb')

Client Scheduler: tcp://127.0.0.1:37909 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 2 Cores: 2 Memory: 11.23 GiB


In [4]:
path = '/g/data/w48/kb6999/Models/SEA_data_for_plots/'
# choose which variable to save as netcdfs
var = 'tas'
# var = 'pr'

In [5]:
# store each section of the directory as a string
institution_dir = '/g/data/lp01/CMIP6/CMIP/'
tas_dir = '/historical/r1i1p1f1/Amon/tas/gr1.5/'
pr_dir = '/historical/r1i1p1f1/Amon/pr/gr1.5/'
print(institution_dir, tas_dir, pr_dir)

/g/data/lp01/CMIP6/CMIP/ /historical/r1i1p1f1/Amon/tas/gr1.5/ /historical/r1i1p1f1/Amon/pr/gr1.5/


In [6]:
# 43 models i'm using in my project, common to both tas and pr
models_43 = ['ACCESS-CM2','ACCESS-ESM1-5','AWI-CM-1-1-MR','AWI-ESM-1-1-LR','BCC-CSM2-MR','BCC-ESM1',
                             'CAMS-CSM1-0', 'CAS-ESM2-0', 'CESM2', 'CESM2-FV2', 'CESM2-WACCM', 'CESM2-WACCM-FV2', 
                             'CMCC-CM2-HR4', 'CMCC-CM2-SR5' ,'CanESM5', 'E3SM-1-1' ,'E3SM-1-1-ECA' ,'EC-Earth3', 
                             'EC-Earth3-AerChem' , 'EC-Earth3-Veg-LR', 'FGOALS-f3-L' ,'FGOALS-g3', 'FIO-ESM-2-0', 
                             'GFDL-CM4' ,'GFDL-ESM4', 'GISS-E2-1-G' ,'GISS-E2-1-G-CC', 'GISS-E2-1-H' ,'IITM-ESM', 
                             'INM-CM4-8', 'INM-CM5-0' ,'IPSL-CM6A-LR', 'KACE-1-0-G', 'MIROC6' ,'MPI-ESM-1-2-HAM',
                             'MPI-ESM1-2-LR' ,'MRI-ESM2-0', 'NESM3', 'NorCPM1' ,'NorESM2-LM', 'NorESM2-MM',
                             'SAM0-UNICON', 'TaiESM1']

In [ ]:
if var == 'tas':
    # read in tas models 
    models_tas = funcM.read_models(institution_dir, tas_dir, '1850-01','2015-01')
    # delete the height from the models_tas array (if it hasn't been deleted already)
    if 'height' in models_tas:
        del models_tas['height']
    # select out the 43 models i'm using for tas and pr 
    models = models_tas.sel(model= models_43)
        
elif var == 'pr':
    # read in pr models
    models_pr = funcM.read_models(institution_dir, pr_dir, '1850-01','2015-01')
    # select out the 43 models i'm using for tas and pr 
    models = models_pr.sel(model= models_43)
    models['pr'] = models.pr*86400

In [ ]:
# sort models alphabetically and prints final model list
models = models.sortby('model')
print('The following', len(models.model.data), 'models will be used in all subsequent calculations: \n')
print(models.model.data)

In [ ]:
# area weighting 
models_w = models*np.cos(models.lat*(np.pi/180))

Right now I have all the models stored in one array so from hereafter I can calculate anomalies etc.  
## Dates and volcano names

In [5]:
# eruption date
e_date = pd.to_datetime(['1883-08','1902-10', '1963-03', '1982-04', '1991-06'])
# e_pre5yrs = ['1878-08','1897-10', '1958-03', '1977-04', '1986-06']
# e_pre3mon = ['1883-05','1902-7', '1962-12', '1982-01', '1991-03']
# e_post5yrs = ['1888-08','1907-10', '1968-03', '1987-04', '1996-06']

In [6]:
# save arrays of times relative to each eruption
e_pre5yrs = e_date - 60 * pd.offsets.MonthBegin()
e_pre3mon = e_date - 3 * pd.offsets.MonthBegin()
e_post5yrs = e_date + 61 * pd.offsets.MonthBegin()

In [7]:
print(e_pre5yrs[1], e_pre3mon[0], e_post5yrs)

1897-10-01 00:00:00 1883-05-01 00:00:00 DatetimeIndex(['1888-09-01', '1907-11-01', '1968-04-01', '1987-05-01',
               '1996-07-01'],
              dtype='datetime64[ns]', freq=None)


In [8]:
# eruption names
volcanoes = ['Krakatoa', 'Santa Maria','Agung','El Chichon', 'Pinatubo']

## Select out regions and eruption dates

In [ ]:
# select out hte 10 year time frame around each eruption (5yrs before and after)
# tas
Krakatoa = models_w.sel(time = slice(e_pre5yrs[0], e_post5yrs[0]))
SantaMaria = models_w.sel(time = slice(e_pre5yrs[1], e_post5yrs[1]))
Agung = models_w.sel(time = slice(e_pre5yrs[2], e_post5yrs[2]))
ElChicon = models_w.sel(time = slice(e_pre5yrs[3], e_post5yrs[3]))
Pinatubo = models_w.sel(time = slice(e_pre5yrs[4], e_post5yrs[4]))

## Anomalies 

In [9]:
# use functions to calculate the monthly anomalies for the globe
K_mon_anom_Glob = func.monthly_anomaly(Krakatoa, e_pre5yrs[0], e_pre3mon[0])
S_mon_anom_Glob = func.monthly_anomaly(SantaMaria, e_pre5yrs[1], e_pre3mon[1])
A_mon_anom_Glob = func.monthly_anomaly(Agung, e_pre5yrs[2], e_pre3mon[2])
E_mon_anom_Glob = func.monthly_anomaly(ElChicon, e_pre5yrs[3], e_pre3mon[3])
P_mon_anom_Glob = func.monthly_anomaly(Pinatubo, e_pre5yrs[4], e_pre3mon[4])

NameError: name 'Krakatoa' is not defined

## save off anomalies as netcdfs 

### don't need to run the next two cos i already savd off the netcdfs, so can just open them below 

In [ ]:
# # save off anoms for SEA plots 
# with dask.diagnostics.ProgressBar():
#     if var == 'tas':
#         K_mon_anom_Glob.tas.to_netcdf(f'{path}K_mon_anom_Glob_{var}.nc')
#     elif var == 'pr':
#         K_mon_anom_Glob.pr.to_netcdf(f'{path}K_mon_anom_Glob_{var}.nc')

In [ ]:
# # save off anoms for SEA plots 
# with dask.diagnostics.ProgressBar():
#     if var == 'tas':
#         S_mon_anom_Glob.tas.to_netcdf(f'{path}S_mon_anom_Glob_{var}.nc')
#         A_mon_anom_Glob.tas.to_netcdf(f'{path}A_mon_anom_Glob_{var}.nc')
#         E_mon_anom_Glob.tas.to_netcdf(f'{path}E_mon_anom_Glob_{var}.nc')
#         P_mon_anom_Glob.tas.to_netcdf(f'{path}P_mon_anom_Glob_{var}.nc')
#     elif var == 'pr':
# #         S_mon_anom_Glob.pr.to_netcdf(f'{path}S_mon_anom_Glob_{var}.nc')
#         A_mon_anom_Glob.pr.to_netcdf(f'{path}A_mon_anom_Glob_{var}.nc')
#         E_mon_anom_Glob.pr.to_netcdf(f'{path}E_mon_anom_Glob_{var}.nc')
#         P_mon_anom_Glob.pr.to_netcdf(f'{path}P_mon_anom_Glob_{var}.nc')

In [5]:
# open anoms 
K_mon_anom_Glob = xr.open_dataset(f'{path}K_mon_anom_Glob_{var}.nc')
S_mon_anom_Glob = xr.open_dataset(f'{path}S_mon_anom_Glob_{var}.nc')
A_mon_anom_Glob = xr.open_dataset(f'{path}A_mon_anom_Glob_{var}.nc')
E_mon_anom_Glob = xr.open_dataset(f'{path}E_mon_anom_Glob_{var}.nc')
P_mon_anom_Glob = xr.open_dataset(f'{path}P_mon_anom_Glob_{var}.nc')

In [6]:
times = np.arange(-60,61)
# reset all the times so 0 corresponds to the eruption year and month
K_mon_anom_Glob['time'] = times
S_mon_anom_Glob['time'] = times
A_mon_anom_Glob['time'] = times
E_mon_anom_Glob['time'] = times 
P_mon_anom_Glob['time'] = times

# delete the months dimension (but not the time)
if 'month' in P_mon_anom_Glob:
    del K_mon_anom_Glob['month']
    del S_mon_anom_Glob['month']
    del A_mon_anom_Glob['month']
    del E_mon_anom_Glob['month']
    del P_mon_anom_Glob['month']

In [7]:
# import land fraction data
landfrac_ds = xr.open_dataset('/home/563/kb6999/Models/landfraction_file_grid1.5.nc')
landmask = landfrac_ds.mean(dim='time')

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-21.04/lib/python3.8/site-packages/xarray/coding/times.py:119: SerializationWarning: Ambiguous reference date string: 1-01-01 01:00:00. The first value is assumed to be the year hence will be padded with zeros to remove the ambiguity (the padded reference date string is: 0001-01-01 01:00:00). To remove this message, remove the ambiguity by padding your reference date strings with zeros.
  warnings.warn(warning_msg, SerializationWarning)
/g/data3/hh5/public/apps/miniconda3/envs/analysis3-21.04/lib/python3.8/site-packages/xarray/coding/times.py:527: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/g/data3/hh5/public/apps/miniconda3/envs/analysis3-21.04/lib/python3.8/site-packages/xarray/core/indexing.py:572: SerializationWarning: Unable to decode time 

## krakatoa 

In [12]:
#  select out each region for each volvnao
K_mon_SH = K_mon_anom_Glob.sel(lat=slice(-90,0))
K_mon_Aus = K_mon_anom_Glob.sel(lat=slice(-50,-10), lon=slice(110,160)).where(landmask.data==1, drop=True)
K_mon_EA = K_mon_anom_Glob.sel(lat=slice(-50,-10), lon=slice(140,155)).where(landmask.data==1, drop=True)

In [13]:
# calculate the monthly model mean for monthly anomalies
K_llm_Glob = K_mon_anom_Glob.mean(dim=('lat','lon'))
K_llm_SH = K_mon_SH.mean(dim=('lat','lon'))
K_llm_Aus = K_mon_Aus.mean(dim=('lat','lon'))
K_llm_EA = K_mon_EA.mean(dim=('lat','lon'))
K_llm_Aus

<xarray.Dataset>
Dimensions:  (model: 43, time: 121)
Coordinates:
  * time     (time) int64 -60 -59 -58 -57 -56 -55 -54 ... 54 55 56 57 58 59 60
  * model    (model) object 'ACCESS-CM2' 'ACCESS-ESM1-5' ... 'TaiESM1'
Data variables:
    tas      (model, time) float64 0.9382 0.527 -0.7236 ... -0.6442 0.4948

In [14]:
# save off llm for SEA plots krakatoa
with dask.diagnostics.ProgressBar():
    if var == 'tas':
        K_llm_Glob.tas.to_netcdf(f'{path}K_llm_Glob_{var}.nc')
        K_llm_SH.tas.to_netcdf(f'{path}K_llm_SH_{var}.nc')
        K_llm_Aus.tas.to_netcdf(f'{path}K_llm_Aus_{var}.nc')
        K_llm_EA.tas.to_netcdf(f'{path}K_llm_EA_{var}.nc')
    elif var == 'pr':
        K_llm_Glob.pr.to_netcdf(f'{path}K_llm_Glob_{var}.nc')
        K_llm_SH.pr.to_netcdf(f'{path}K_llm_SH_{var}.nc')
        K_llm_Aus.pr.to_netcdf(f'{path}K_llm_Aus_{var}.nc')
        K_llm_EA.pr.to_netcdf(f'{path}K_llm_EA_{var}.nc')

In [ ]:
# calculate the monthly model mean for monthly anomalies
K_mmm_Glob = K_mon_anom_Glob.mean(dim='model').mean(dim=('lat','lon'))
K_mmm_SH = K_mon_SH.mean(dim='model').mean(dim=('lat','lon'))
K_mmm_Aus = K_mon_Aus.mean(dim='model').mean(dim=('lat','lon'))
K_mmm_EA = K_mon_EA.mean(dim='model').mean(dim=('lat','lon'))
K_mmm_Aus

In [ ]:
# save off llm mmm for SEA plots krakatoa
with dask.diagnostics.ProgressBar():
    if var == 'tas':
        K_mmm_Glob.tas.to_netcdf(f'{path}K_mmm_Glob_{var}.nc')
        K_mmm_SH.tas.to_netcdf(f'{path}K_mmm_SH_{var}.nc')
        K_mmm_Aus.tas.to_netcdf(f'{path}K_mmm_Aus_{var}.nc')
        K_mmm_EA.tas.to_netcdf(f'{path}K_mmm_EA_{var}.nc')
    elif var == 'pr':
        K_mmm_Glob.pr.to_netcdf(f'{path}K_mmm_Glob_{var}.nc')
        K_mmm_SH.pr.to_netcdf(f'{path}K_mmm_SH_{var}.nc')
        K_mmm_Aus.pr.to_netcdf(f'{path}K_mmm_Aus_{var}.nc')
        K_mmm_EA.pr.to_netcdf(f'{path}K_mmm_EA_{var}.nc')

## santa maria

In [12]:
#  select out each region for eac
S_mon_SH = S_mon_anom_Glob.sel(lat=slice(-90,0))
S_mon_Aus = S_mon_anom_Glob.sel(lat=slice(-50,-10), lon=slice(110,160)).where(landmask.data==1, drop=True)
S_mon_EA = S_mon_anom_Glob.sel(lat=slice(-50,-10), lon=slice(140,155)).where(landmask.data==1, drop=True)

In [13]:
# calculate the monthly model mean for monthly anomalies
S_llm_Glob = S_mon_anom_Glob.mean(dim=('lat','lon'))
S_llm_SH = S_mon_SH.mean(dim=('lat','lon'))
S_llm_Aus = S_mon_Aus.mean(dim=('lat','lon'))
S_llm_EA = S_mon_EA.mean(dim=('lat','lon'))
S_llm_Aus

<xarray.Dataset>
Dimensions:  (model: 43, time: 121)
Coordinates:
  * time     (time) int64 -60 -59 -58 -57 -56 -55 -54 ... 54 55 56 57 58 59 60
  * model    (model) object 'ACCESS-CM2' 'ACCESS-ESM1-5' ... 'TaiESM1'
Data variables:
    tas      (model, time) float64 0.715 -0.03689 0.536 ... 0.7583 0.1736 -1.809

In [14]:
# save off llm for SEA plots krakatoa
with dask.diagnostics.ProgressBar():
    if var == 'tas':
        S_llm_Glob.tas.to_netcdf(f'{path}S_llm_Glob_{var}.nc')
        S_llm_SH.tas.to_netcdf(f'{path}S_llm_SH_{var}.nc')
        S_llm_Aus.tas.to_netcdf(f'{path}S_llm_Aus_{var}.nc')
        S_llm_EA.tas.to_netcdf(f'{path}S_llm_EA_{var}.nc')
    elif var == 'pr':
        S_llm_Glob.pr.to_netcdf(f'{path}S_llm_Glob_{var}.nc')
        S_llm_SH.pr.to_netcdf(f'{path}S_llm_SH_{var}.nc')
        S_llm_Aus.pr.to_netcdf(f'{path}S_llm_Aus_{var}.nc')
        S_llm_EA.pr.to_netcdf(f'{path}S_llm_EA_{var}.nc')

In [14]:
# calculate the monthly model mean for monthly anomalies
S_mmm_Glob = S_mon_anom_Glob.mean(dim='model').mean(dim=('lat','lon'))
S_mmm_SH = S_mon_SH.mean(dim='model').mean(dim=('lat','lon'))
S_mmm_Aus = S_mon_Aus.mean(dim='model').mean(dim=('lat','lon'))
S_mmm_EA = S_mon_EA.mean(dim='model').mean(dim=('lat','lon'))
S_mmm_Aus

<xarray.Dataset>
Dimensions:  (time: 121)
Coordinates:
  * time     (time) int64 -60 -59 -58 -57 -56 -55 -54 ... 54 55 56 57 58 59 60
Data variables:
    pr       (time) float64 -0.01563 0.02718 0.0006509 ... 0.07275 0.1287

In [15]:
# save off llm mmm for SEA plots santa maria
with dask.diagnostics.ProgressBar():
    if var == 'tas':
        S_mmm_Glob.tas.to_netcdf(f'{path}S_mmm_Glob_{var}.nc')
        S_mmm_SH.tas.to_netcdf(f'{path}S_mmm_SH_{var}.nc')
        S_mmm_Aus.tas.to_netcdf(f'{path}S_mmm_Aus_{var}.nc')
        S_mmm_EA.tas.to_netcdf(f'{path}S_mmm_EA_{var}.nc')
    elif var == 'pr':
        S_mmm_Glob.pr.to_netcdf(f'{path}S_mmm_Glob_{var}.nc')
        S_mmm_SH.pr.to_netcdf(f'{path}S_mmm_SH_{var}.nc')
        S_mmm_Aus.pr.to_netcdf(f'{path}S_mmm_Aus_{var}.nc')
        S_mmm_EA.pr.to_netcdf(f'{path}S_mmm_EA_{var}.nc')

## agung

In [12]:
#  select out each region for eac
A_mon_SH = A_mon_anom_Glob.sel(lat=slice(-90,0))
A_mon_Aus = A_mon_anom_Glob.sel(lat=slice(-50,-10), lon=slice(110,160)).where(landmask.data==1, drop=True)
A_mon_EA = A_mon_anom_Glob.sel(lat=slice(-50,-10), lon=slice(140,155)).where(landmask.data==1, drop=True)

In [13]:
# calculate the monthly model mean for monthly anomalies
A_llm_Glob = A_mon_anom_Glob.mean(dim=('lat','lon'))
A_llm_SH = A_mon_SH.mean(dim=('lat','lon'))
A_llm_Aus = A_mon_Aus.mean(dim=('lat','lon'))
A_llm_EA = A_mon_EA.mean(dim=('lat','lon'))
A_llm_Aus

<xarray.Dataset>
Dimensions:  (model: 43, time: 121)
Coordinates:
  * time     (time) int64 -60 -59 -58 -57 -56 -55 -54 ... 54 55 56 57 58 59 60
  * model    (model) object 'ACCESS-CM2' 'ACCESS-ESM1-5' ... 'TaiESM1'
Data variables:
    tas      (model, time) float64 0.3763 0.5238 0.2524 ... -1.348 0.1116

In [14]:
# save off llm for SEA plots krakatoa
with dask.diagnostics.ProgressBar():
    if var == 'tas':
        A_llm_Glob.tas.to_netcdf(f'{path}A_llm_Glob_{var}.nc')
        A_llm_SH.tas.to_netcdf(f'{path}A_llm_SH_{var}.nc')
        A_llm_Aus.tas.to_netcdf(f'{path}A_llm_Aus_{var}.nc')
        A_llm_EA.tas.to_netcdf(f'{path}A_llm_EA_{var}.nc')
    elif var == 'pr':
        A_llm_Glob.pr.to_netcdf(f'{path}A_llm_Glob_{var}.nc')
        A_llm_SH.pr.to_netcdf(f'{path}A_llm_SH_{var}.nc')
        A_llm_Aus.pr.to_netcdf(f'{path}A_llm_Aus_{var}.nc')
        A_llm_EA.pr.to_netcdf(f'{path}A_llm_EA_{var}.nc')

In [13]:
# calculate the monthly model mean for monthly anomalies
A_mmm_Glob = A_mon_anom_Glob.mean(dim='model').mean(dim=('lat','lon'))
A_mmm_SH = A_mon_SH.mean(dim='model').mean(dim=('lat','lon'))
A_mmm_Aus = A_mon_Aus.mean(dim='model').mean(dim=('lat','lon'))
A_mmm_EA = A_mon_EA.mean(dim='model').mean(dim=('lat','lon'))
A_mmm_Aus

<xarray.Dataset>
Dimensions:  (time: 121)
Coordinates:
  * time     (time) int64 -60 -59 -58 -57 -56 -55 -54 ... 54 55 56 57 58 59 60
Data variables:
    pr       (time) float64 0.0002863 0.07069 0.03208 ... -0.1756 -0.08233

In [14]:
# save off llm mmm for SEA plots agung
with dask.diagnostics.ProgressBar():
    if var == 'tas':
        A_mmm_Glob.tas.to_netcdf(f'{path}A_mmm_Glob_{var}.nc')
        A_mmm_SH.tas.to_netcdf(f'{path}A_mmm_SH_{var}.nc')
        A_mmm_Aus.tas.to_netcdf(f'{path}A_mmm_Aus_{var}.nc')
        A_mmm_EA.tas.to_netcdf(f'{path}A_mmm_EA_{var}.nc')
    elif var == 'pr':
        A_mmm_Glob.pr.to_netcdf(f'{path}A_mmm_Glob_{var}.nc')
        A_mmm_SH.pr.to_netcdf(f'{path}A_mmm_SH_{var}.nc')
        A_mmm_Aus.pr.to_netcdf(f'{path}A_mmm_Aus_{var}.nc')
        A_mmm_EA.pr.to_netcdf(f'{path}A_mmm_EA_{var}.nc')

## el chichon

In [12]:
#  select out each region for eac
E_mon_SH = E_mon_anom_Glob.sel(lat=slice(-90,0))
E_mon_Aus = E_mon_anom_Glob.sel(lat=slice(-50,-10), lon=slice(110,160)).where(landmask.data==1, drop=True)
E_mon_EA = E_mon_anom_Glob.sel(lat=slice(-50,-10), lon=slice(140,155)).where(landmask.data==1, drop=True)

In [13]:
# calculate the monthly model mean for monthly anomalies
E_llm_Glob = E_mon_anom_Glob.mean(dim=('lat','lon'))
E_llm_SH = E_mon_SH.mean(dim=('lat','lon'))
E_llm_Aus = E_mon_Aus.mean(dim=('lat','lon'))
E_llm_EA = E_mon_EA.mean(dim=('lat','lon'))
E_llm_Aus

<xarray.Dataset>
Dimensions:  (model: 43, time: 121)
Coordinates:
  * time     (time) int64 -60 -59 -58 -57 -56 -55 -54 ... 54 55 56 57 58 59 60
  * model    (model) object 'ACCESS-CM2' 'ACCESS-ESM1-5' ... 'TaiESM1'
Data variables:
    tas      (model, time) float64 0.8758 1.41 0.1287 ... -0.5571 -0.7952 0.2338

In [14]:
# save off llm for SEA plots krakatoa
with dask.diagnostics.ProgressBar():
    if var == 'tas':
        E_llm_Glob.tas.to_netcdf(f'{path}E_llm_Glob_{var}.nc')
        E_llm_SH.tas.to_netcdf(f'{path}E_llm_SH_{var}.nc')
        E_llm_Aus.tas.to_netcdf(f'{path}E_llm_Aus_{var}.nc')
        E_llm_EA.tas.to_netcdf(f'{path}E_llm_EA_{var}.nc')
    elif var == 'pr':
        E_llm_Glob.pr.to_netcdf(f'{path}E_llm_Glob_{var}.nc')
        E_llm_SH.pr.to_netcdf(f'{path}E_llm_SH_{var}.nc')
        E_llm_Aus.pr.to_netcdf(f'{path}E_llm_Aus_{var}.nc')
        E_llm_EA.pr.to_netcdf(f'{path}E_llm_EA_{var}.nc')

In [13]:
# calculate the monthly model mean for monthly anomalies
E_mmm_Glob = E_mon_anom_Glob.mean(dim='model').mean(dim=('lat','lon'))
E_mmm_SH = E_mon_SH.mean(dim='model').mean(dim=('lat','lon'))
E_mmm_Aus = E_mon_Aus.mean(dim='model').mean(dim=('lat','lon'))
E_mmm_EA = E_mon_EA.mean(dim='model').mean(dim=('lat','lon'))
E_mmm_Aus

<xarray.Dataset>
Dimensions:  (time: 121)
Coordinates:
  * time     (time) int64 -60 -59 -58 -57 -56 -55 -54 ... 54 55 56 57 58 59 60
Data variables:
    pr       (time) float64 -0.1508 -0.05844 0.004157 ... -0.2783 0.3912 -0.0385

In [14]:
# save off llm mmm for SEA plots el chichon
with dask.diagnostics.ProgressBar():
    if var == 'tas':
        E_mmm_Glob.tas.to_netcdf(f'{path}E_mmm_Glob_{var}.nc')
        E_mmm_SH.tas.to_netcdf(f'{path}E_mmm_SH_{var}.nc')
        E_mmm_Aus.tas.to_netcdf(f'{path}E_mmm_Aus_{var}.nc')
        E_mmm_EA.tas.to_netcdf(f'{path}E_mmm_EA_{var}.nc')
    elif var == 'pr':
        E_mmm_Glob.pr.to_netcdf(f'{path}E_mmm_Glob_{var}.nc')
        E_mmm_SH.pr.to_netcdf(f'{path}E_mmm_SH_{var}.nc')
        E_mmm_Aus.pr.to_netcdf(f'{path}E_mmm_Aus_{var}.nc')
        E_mmm_EA.pr.to_netcdf(f'{path}E_mmm_EA_{var}.nc')


## pinatubo

In [8]:
#  select out each region for eac
P_mon_SH = P_mon_anom_Glob.sel(lat=slice(-90,0))
P_mon_Aus = P_mon_anom_Glob.sel(lat=slice(-50,-10), lon=slice(110,160)).where(landmask.data==1, drop=True)
P_mon_EA = P_mon_anom_Glob.sel(lat=slice(-50,-10), lon=slice(140,155)).where(landmask.data==1, drop=True)

In [9]:
# calculate the monthly model mean for monthly anomalies
P_llm_Glob = P_mon_anom_Glob.mean(dim=('lat','lon'))
P_llm_SH = P_mon_SH.mean(dim=('lat','lon'))
P_llm_Aus = P_mon_Aus.mean(dim=('lat','lon'))
P_llm_EA = P_mon_EA.mean(dim=('lat','lon'))
P_llm_Aus

<xarray.Dataset>
Dimensions:  (model: 43, time: 121)
Coordinates:
  * time     (time) int64 -60 -59 -58 -57 -56 -55 -54 ... 54 55 56 57 58 59 60
  * model    (model) object 'ACCESS-CM2' 'ACCESS-ESM1-5' ... 'TaiESM1'
Data variables:
    tas      (model, time) float64 -1.123 0.4511 -0.5525 ... -1.591 -1.09 -0.436

In [10]:
# save off llm for SEA plots krakatoa
with dask.diagnostics.ProgressBar():
    if var == 'tas':
        P_llm_Glob.tas.to_netcdf(f'{path}P_llm_Glob_{var}.nc')
        P_llm_SH.tas.to_netcdf(f'{path}P_llm_SH_{var}.nc')
        P_llm_Aus.tas.to_netcdf(f'{path}P_llm_Aus_{var}.nc')
        P_llm_EA.tas.to_netcdf(f'{path}P_llm_EA_{var}.nc')
    elif var == 'pr':
        P_llm_Glob.pr.to_netcdf(f'{path}P_llm_Glob_{var}.nc')
        P_llm_SH.pr.to_netcdf(f'{path}P_llm_SH_{var}.nc')
        P_llm_Aus.pr.to_netcdf(f'{path}P_llm_Aus_{var}.nc')
        P_llm_EA.pr.to_netcdf(f'{path}P_llm_EA_{var}.nc')

In [13]:
# calculate the monthly model mean for monthly anomalies
P_mmm_Glob = P_mon_anom_Glob.mean(dim='model').mean(dim=('lat','lon'))
P_mmm_SH = P_mon_SH.mean(dim='model').mean(dim=('lat','lon'))
P_mmm_Aus = P_mon_Aus.mean(dim='model').mean(dim=('lat','lon'))
P_mmm_EA = P_mon_EA.mean(dim='model').mean(dim=('lat','lon'))
P_mmm_Aus

<xarray.Dataset>
Dimensions:  (time: 121)
Coordinates:
  * time     (time) int64 -60 -59 -58 -57 -56 -55 -54 ... 54 55 56 57 58 59 60
Data variables:
    pr       (time) float64 0.194 0.08357 0.2175 ... 0.002293 0.02728 -0.02611

In [14]:
# save off llm mmm for SEA plots krakatoa
with dask.diagnostics.ProgressBar():
    if var == 'tas':
        P_mmm_Glob.tas.to_netcdf(f'{path}P_mmm_Glob_{var}.nc')
        P_mmm_SH.tas.to_netcdf(f'{path}P_mmm_SH_{var}.nc')
        P_mmm_Aus.tas.to_netcdf(f'{path}P_mmm_Aus_{var}.nc')
        P_mmm_EA.tas.to_netcdf(f'{path}P_mmm_EA_{var}.nc')
    elif var == 'pr':
        P_mmm_Glob.pr.to_netcdf(f'{path}P_mmm_Glob_{var}.nc')
        P_mmm_SH.pr.to_netcdf(f'{path}P_mmm_SH_{var}.nc')
        P_mmm_Aus.pr.to_netcdf(f'{path}P_mmm_Aus_{var}.nc')
        P_mmm_EA.pr.to_netcdf(f'{path}P_mmm_EA_{var}.nc')

In [ ]:
# # select out each region
# composite_mon_SH = composite_mon_Glob.sel(lat=slice(-90,0))
# composite_mon_Aus = composite_mon_Glob.sel(lat=slice(-50,-10), lon=slice(110,160)).where(landmask.data==1, drop=True)
# composite_mon_EA = composite_mon_Glob.sel(lat=slice(-50,-10), lon=slice(140,155)).where(landmask.data==1, drop=True)